If I add a nested column (ping_info.end_time) it causes the kernel to restart.

In [1]:
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession

2021-09-29 22:35:25.411909: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


In [2]:
GCP_PROJECT_ID = 'dataorg-hackweek-2021'
DATASET_GCP_PROJECT_ID = GCP_PROJECT_ID 
DATASET_ID = 'mlhackweek_search.action'

In [3]:
def features_and_labels(features):
  label = features.pop('document_id') # this is what we will train for
  return features, label

def read_dataset(client, row_restriction, batch_size=2048):
    COL_NAMES = ["document_id", "normalized_app_name"]
   
    COL_TYPES = [dtypes.string, dtypes.string]
    DATASET_GCP_PROJECT_ID, DATASET_ID, TABLE_ID,  = 'moz-fx-data-shared-prod.mlhackweek_search_stable.action_v1'.split('.')
    print(f"GCP_PROJECT_ID: {GCP_PROJECT_ID}")
    print(f"DATASET_GCP_PROJECT_ID: {DATASET_GCP_PROJECT_ID}")
    print(f"DATASET_ID: {DATASET_ID}")
    print(f"TABLE_ID: {TABLE_ID}")
    
    bqsession = client.read_session(
        parent="projects/" + GCP_PROJECT_ID,
        project_id=DATASET_GCP_PROJECT_ID, 
        table_id=TABLE_ID, 
        dataset_id=DATASET_ID,
        selected_fields=COL_NAMES, 
        output_types=COL_TYPES,
        requested_streams=2,
        row_restriction=row_restriction)
    dataset = bqsession.parallel_read_rows()
    return dataset.prefetch(1).map(features_and_labels).shuffle(batch_size*10).batch(batch_size)

client = BigQueryClient()

2021-09-29 22:35:25.566095: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200160000 Hz
2021-09-29 22:35:25.566489: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55bbbfca1860 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-29 22:35:25.566517: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-29 22:35:25.566668: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
temp_df = read_dataset(client, 'DATE(submission_timestamp) >= DATE("2021-01-19")', 2)
for row in temp_df:
    print(row)
    break

GCP_PROJECT_ID: dataorg-hackweek-2021
DATASET_GCP_PROJECT_ID: moz-fx-data-shared-prod
DATASET_ID: mlhackweek_search_stable
TABLE_ID: action_v1
(OrderedDict([('normalized_app_name', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'', b''], dtype=object)>)]), <tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'db003f3d-3478-440c-8a3b-26419cd206a7',
       b'cf07cf48-931d-4741-8d80-63cb11dc0a8b'], dtype=object)>)
